# Ocean Oxygen Changes in CMIP6
In response to Issue 1 proposed by Matt Long (https://discourse.pangeo.io/t/ocean-oxygen-in-a-warming-world/68), this notebook computes the change in apparent oxygen utilization (AOU) and oxygen saturation in the top 1000 meters over time. The signal is then split up into regional trends.

Shawnee Traylor, MIT-WHOI

In [2]:
# Set up environment and load collection
%matplotlib inline

from itertools import product
import numpy as np
import pandas as pd
import xarray as xr
import intake
import matplotlib.pyplot as plt
# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 

# Load collection
if util.is_ncar_host():
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")

import pprint 
uni_dict = col.unique(['source_id', 'experiment_id','table_id'])
pprint.pprint(uni_dict, compact=True)

/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


{'experiment_id': {'count': 82,
                   'values': ['ssp370', 'histSST-piNTCF', 'histSST',
                              'histSST-1950HC', 'hist-1950HC', 'hist-piNTCF',
                              'piClim-NTCF', 'ssp370SST-lowNTCF',
                              'ssp370-lowNTCF', 'ssp370SST', '1pctCO2-bgc',
                              'hist-bgc', 'esm-ssp585', '1pctCO2-cdr',
                              'amip-future4K', 'amip-m4K', 'a4SST', 'aqua-p4K',
                              'piSST', 'amip-4xCO2', 'a4SSTice', 'amip-p4K',
                              'aqua-control', 'aqua-4xCO2', 'abrupt-4xCO2',
                              'historical', 'piControl', 'amip', '1pctCO2',
                              'esm-hist', 'esm-piControl', 'ssp245', 'ssp585',
                              'ssp126', 'hist-GHG', 'hist-aer', 'hist-nat',
                              'dcppA-hindcast', 'dcppC-hindcast-noPinatubo',
                              'dcppC-hindcast-noElChichon', 'dcppA-

In [3]:
# Choose experiments
experiments = ['historical', 'ssp585']

def get_models(table_id, variable_id):
    # all models
    models = set(uni_dict['source_id']['values'])

    for experiment_id in experiments:
        query = dict(experiment_id=experiment_id, variable_id=variable_id, 
                     table_id=table_id, grid_label='gn')  
        cat = col.search(**query)
        models = models.intersection({model for model in cat.df.source_id.unique().tolist()})

    # ensure the CESM2 models are not included (oxygen was erroneously submitted to the archive)
    return models - {'CESM2-WACCM', 'CESM2'}
    

models = {}    

# find models with O2sat, the less common variable
models['Omon.o2sat'] = get_models('Omon', 'o2sat')
have_sat = models['Omon.o2sat']

# find models with O2sat that also have O2, the more common variable
models['Omon.o2'] = get_models('Omon', 'o2').intersection(have_sat)

models_all = list(models['Omon.o2'])
models

{'Omon.o2sat': {'UKESM1-0-LL'}, 'Omon.o2': {'UKESM1-0-LL'}}

In [4]:
# Create dataframe
df = pd.DataFrame()

# Load in models list and pull out associated data
for key, val in models.items():
    model_list = list(val)
    table_id = key.split('.')[0]
    variable_id = key.split('.')[1]
    
    cat = col.search(experiment_id=experiments, table_id=table_id, 
                     variable_id=variable_id, 
                     source_id=model_list, grid_label='gn')
    df = pd.concat((df, cat.df))

cat.df = df.copy()
cat.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
204404,CMIP,MOHC,UKESM1-0-LL,historical,r3i1p1f2,Omon,o2sat,gn,NaN,v20190708,200001-201412,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
204405,CMIP,MOHC,UKESM1-0-LL,historical,r3i1p1f2,Omon,o2sat,gn,NaN,v20190708,185001-189912,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
205771,CMIP,MOHC,UKESM1-0-LL,historical,r4i1p1f2,Omon,o2sat,gn,NaN,v20190708,195001-199912,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
205772,CMIP,MOHC,UKESM1-0-LL,historical,r4i1p1f2,Omon,o2sat,gn,NaN,v20190708,185001-189912,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
206606,CMIP,MOHC,UKESM1-0-LL,historical,r8i1p1f2,Omon,o2sat,gn,NaN,v20190708,185001-189912,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...


Only one model included oxygen saturation and oxygen concentration, so this model was isolated. Only one member_id was chosen for simplicity. Future versions should enable more member_ids and calculated oxygen saturations in order to include more models.

In [5]:
df_Omon_o2_sat = df[(cat.df.table_id == 'Omon') & (cat.df.source_id == 'UKESM1-0-LL') & (cat.df.member_id == 'r1i1p1f2') & (cat.df.variable_id == 'o2sat')]
df_Omon_o2 = df[(cat.df.table_id == 'Omon') & (cat.df.source_id == 'UKESM1-0-LL') & (cat.df.member_id == 'r1i1p1f2') & (cat.df.variable_id == 'o2')]

df = pd.concat((df_Omon_o2, df_Omon_o2_sat))

cat.df = df.copy()
cat.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
207549,CMIP,MOHC,UKESM1-0-LL,historical,r1i1p1f2,Omon,o2,gn,NaN,v20190627,195001-199912,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
207550,CMIP,MOHC,UKESM1-0-LL,historical,r1i1p1f2,Omon,o2,gn,NaN,v20190627,190001-194912,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
1100664,ScenarioMIP,MOHC,UKESM1-0-LL,ssp585,r1i1p1f2,Omon,o2,gn,NaN,v20190726,201501-204912,/glade/collections/cmip/CMIP6/ScenarioMIP/MOHC...
1100665,ScenarioMIP,MOHC,UKESM1-0-LL,ssp585,r1i1p1f2,Omon,o2,gn,NaN,v20190726,205001-209912,/glade/collections/cmip/CMIP6/ScenarioMIP/MOHC...
1100666,ScenarioMIP,MOHC,UKESM1-0-LL,ssp585,r1i1p1f2,Omon,o2,gn,NaN,v20190726,210001-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MOHC...


In [6]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': True}, 
                                cdf_kwargs={'chunks': {'time': 48}, 'decode_times': True})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 2 group(s)


In [7]:
dset_dict.keys()

dict_keys(['CMIP.MOHC.UKESM1-0-LL.historical.Omon.gn', 'ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Omon.gn'])

In [8]:
cat_fx = col.search(source_id='UKESM1-0-LL', table_id='Ofx', grid_label='gn')
# specify a list of queries to eliminate
corrupt_data = [dict(variable_id='areacello', source_id='IPSL-CM6A-LR',
                     experiment_id='historical', member_id='r2i1p1f1')
               ]

# copy the dataframe 
df = cat_fx.df.copy()

# eliminate data
for elim in corrupt_data:
    condition = np.ones(len(df), dtype=bool)
    for key, val in elim.items():
        condition = condition & (df[key] == val)
    df = df.loc[~condition]

df.drop_duplicates(subset=['source_id', 'variable_id'], inplace=True)
df['member_id'] = np.nan
cat_fx.df = df
df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
210700,CMIP,MOHC,UKESM1-0-LL,piControl,NaN,Ofx,sftof,gn,NaN,v20190705,NaN,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
210701,CMIP,MOHC,UKESM1-0-LL,piControl,NaN,Ofx,areacello,gn,NaN,v20190705,NaN,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
210702,CMIP,MOHC,UKESM1-0-LL,piControl,NaN,Ofx,hfgeou,gn,NaN,v20190705,NaN,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
210703,CMIP,MOHC,UKESM1-0-LL,piControl,NaN,Ofx,basin,gn,NaN,v20190705,NaN,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...
210704,CMIP,MOHC,UKESM1-0-LL,piControl,NaN,Ofx,deptho,gn,NaN,v20190705,NaN,/glade/collections/cmip/CMIP6/CMIP/MOHC/UKESM1...


In [9]:
fx_dsets = cat_fx.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False}, 
                                  cdf_kwargs={'chunks': {}, 'decode_times': False})
fx_dsets.keys()


xarray will load netCDF datasets with dask using a single chunk for all arrays.
For effective chunking, please provide chunks in cdf_kwargs.
For example: cdf_kwargs={'chunks': {'time': 36}}

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)


dict_keys(['CMIP.MOHC.UKESM1-0-LL.piControl.Ofx.gn'])

In [10]:
for key, ds in fx_dsets.items():
    print(key)
    print(ds.data_vars)
    print()

CMIP.MOHC.UKESM1-0-LL.piControl.Ofx.gn
Data variables:
    latitude            (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    longitude           (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float32 dask.array<chunksize=(330, 360, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float32 dask.array<chunksize=(330, 360, 4), meta=np.ndarray>
    type                |S3 ...
    sftof               (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    areacello           (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    hfgeou              (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    basin               (j, i) float64 dask.array<chunksize=(330, 360), meta=np.ndarray>
    deptho              (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>



Separate historical and ssp585 using the dicts.

In [11]:
historical = dset_dict['CMIP.MOHC.UKESM1-0-LL.historical.Omon.gn']
ssp585 = dset_dict['ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Omon.gn']

In [12]:
# Create initial conditions. Here we choose 1970. Then concatenate with the projection data from ssp585.
time_0 = historical.sel(time = '1970-01-16')
time_1970_to_1999 = historical.sel(time = slice('1970-01-16', '1999-12-16'))
all_time = xr.concat([time_1970_to_1999,ssp585], dim='time')

# Calculate depth-integrated oxygen

In [13]:
# Choose the top 1000 meters
lev_bnds_1000 = all_time['lev_bnds'].where(all_time['lev'] <= 1000, drop = True)
dz = lev_bnds_1000.isel(bnds=1) - lev_bnds_1000.isel(bnds=0)

# Pull out cell area
area = fx_dsets['CMIP.MOHC.UKESM1-0-LL.piControl.Ofx.gn']['areacello']

In [14]:
# Compute integrals over 1000 m
o2_1000_int = (all_time['o2']*dz).sum('lev')

o2_sat = all_time['o2sat']*1e3 # saturation values appear to still be in mmol/m3, so *1e3
o2_sat_int = (o2_sat*dz).sum('lev')

# Calculate AOU at each time
all_time_aou = all_time.o2sat*1e3-all_time.o2  #saturation values appear to still be in mmol/m3, so *1e3
aou_int = (all_time_aou*dz).sum('lev')

In [15]:
# Compute differences between 1970 and 2100
o2_1000_diff = o2_1000_int.isel(time=-1) - o2_1000_int.isel(time=0)
o2_sat_1000_diff = o2_sat_int.isel(time=-1) - o2_sat_int.isel(time=0)
aou_1000_diff = aou_int.isel(time=-1) - aou_int.isel(time=0)

In [ ]:
# Plots integrated O2 to 1000m (units: mol/m^2)
fig, (axes) = plt.subplots(figsize=(15,15),nrows=3,ncols=2)
o2_1000_int.isel(time=-1).plot(ax=axes[0,0])
o2_1000_diff.plot(ax=axes[0,1])
o2_sat_int.isel(time=-1).plot(ax=axes[1,0])
o2_sat_1000_diff.plot(ax=axes[1,1])
aou_int.isel(time=-1).plot(ax=axes[2,0])
aou_1000_diff.plot(ax=axes[2,1])
     
axes[0,0].set_title("1000 m Integrated Oxygen Concentration in 2100 [mol/m2]")
axes[0,1].set_title("Difference in O2 between 1970-2100 [mol/m2]")
axes[1,0].set_title("1000 m Integrated Oxygen Saturation in 2100 [mol/m2]")
axes[1,1].set_title("Difference in O2sat between 1970-2100 [mol/m2]")
axes[2,0].set_title("1000 m Integrated AOU in 2100 [mol/m2]")
axes[2,1].set_title("Difference in AOU between 1970-2100 [mol/m2]")

for jj in range(0,2):
    for ii in range(0,3):
        axes[ii,jj].set_xlabel("longitude")
        axes[ii,jj].set_ylabel("latitude")

plt.show()

Compute global and regional averages of depth-integrated values.

In [ ]:
# Define tropics (+/- 23.5 degrees) and extratropics
tropics_o2_sat = o2_sat_int.where((o2_sat_int['j'] >= 141.5) & (o2_sat_int['j'] <= 188.5), other = float('NaN'))
tropics_aou = aou_int.where((o2_sat_int['j'] >= 141.5) & (o2_sat_int['j'] <= 188.5), other = float('NaN'))


extratropics_o2_sat = o2_sat_int.where((o2_sat_int['j'] < 141.5) | (o2_sat_int['j'] > 188.5), other = float('NaN'))
extratropics_aou = aou_int.where((o2_sat_int['j'] < 141.5) | (o2_sat_int['j'] > 188.5), other = float('NaN'))

In [ ]:
# Global average per year
global_mean_o2sat = ((o2_sat_int*area).sum(['j','i'])/area.sum(['j','i'])).compute()
global_mean_aou = ((aou_int*area).sum(['j','i'])/area.sum(['j','i'])).compute()

# Difference from 1970
global_delta_o2_sat = global_mean_o2sat - global_mean_o2sat.isel(time = 0)
global_delta_aou = global_mean_aou - global_mean_aou.isel(time = 0)

# Compute annual trend from monthly
annual_delta_o2_sat = global_delta_o2_sat.groupby('time.year').mean('time')
annual_delta_aou = global_delta_aou.groupby('time.year').mean('time')

In [ ]:
# Tropical average per year
tropics_mean_o2_sat = ((tropics_o2_sat*area).sum(['j','i'])/area.sum(['j','i'])).compute()
tropics_mean_aou = ((tropics_aou*area).sum(['j','i'])/area.sum(['j','i'])).compute()

# Difference from 1970
tropics_delta_aou = tropics_mean_aou - tropics_mean_aou.isel(time = 0)
tropics_delta_o2_sat = tropics_mean_o2_sat - tropics_mean_o2_sat.isel(time = 0)

# Compute annual trend from monthly
annual_tropics_delta_aou = tropics_delta_aou.groupby('time.year').mean('time')
annual_tropics_delta_o2_sat = tropics_delta_o2_sat.groupby('time.year').mean('time')

In [ ]:
# Extratropical average per year
extratropics_mean_aou = ((extratropics_aou*area).sum(['j','i'])/area.sum(['j','i'])).compute()
extratropics_mean_o2_sat = ((extratropics_o2_sat*area).sum(['j','i'])/area.sum(['j','i'])).compute()

# Difference from 1970
extratropics_delta_aou = extratropics_mean_aou - extratropics_mean_aou.isel(time = 0)
extratropics_delta_o2_sat = extratropics_mean_o2_sat - extratropics_mean_o2_sat.isel(time = 0)

# Compute annual trend from monthly
annual_extratropics_delta_aou = extratropics_delta_aou.groupby('time.year').mean('time')
annual_extratropics_delta_o2_sat = extratropics_delta_o2_sat.groupby('time.year').mean('time')

In [ ]:
# Plots of AOU vs O2sat
plt.figure()
plt.plot(np.array(annual_delta_o2_sat)[0],np.array(annual_delta_aou)[0])
plt.xlabel("Δ O2 Saturation [mol/m$^3$]")
plt.ylabel('Δ AOU [mol/m$^3$]')
plt.title('Global')

plt.figure() 
plt.plot(np.array(annual_tropics_delta_o2_sat)[0],np.array(annual_tropics_delta_aou)[0])
plt.title('Tropics')
plt.xlabel("Δ O2 Saturation [mol/m$^3$]")
plt.ylabel('Δ AOU [mol/m$^3$]')

plt.figure()
fig = plt.plot(np.array(annual_extratropics_delta_o2_sat)[0],np.array(annual_extratropics_delta_aou)[0])
plt.title('Extratropics')
plt.xlabel("Δ O2 Saturation [mol/m$^3$]")
plt.ylabel('Δ AOU [mol/m$^3$]')